In [55]:
rules= {
    "users": [
      {
        "type": "admin",
        "two_factor_authentication": True,
        "multi_factor_authentication": True,
        "security_monitoring": True,
        "data_privacy_policy": True,
        "secure_file_uploads": True,
        "secure_file_uploads_policies": {
          "properties": {
            "secure_file_name": [".txt", ".csv", ".xlsx", ".pdf",".img",".png",".jpeg",".mp4"],
            "malware_scan": True,
            "audit_logging": True,
            "encryption": {
              "in_transit": True,
              "at_rest": True
            }
          }
        },
        "ssl_encryption_required": True,
        "permissions": ["read", "write", "delete", "create"],
        "explicite_allowed_resources": [
          "sensitve_data.txt",
          "sales.txt",
          "reports.txt",
          "product_info.txt"
        ],
        "other_resources": True
      },
      {
        "type": "employee",
  
        "two_factor_authentication": True,
        "multi_factor_authentication": False,
        "security_monitoring": True,
        "data_privacy_policy": True,
        "secure_file_uploads": True,
        "secure_file_uploads_policies": {
          "properties": {
            "secure_file_name": [".txt", ".csv", ".xlsx", ".pdf",".img",".png",".jpeg",".mp4"],
            "malware_scan": True,
            "audit_logging": True,
            "encryption": {
              "in_transit": True,
              "at_rest": True
            }
          }
        },
        "ssl_encryption_required": True,
        "permissions": ["read", "write", "create"],
        "explicite_allowed_resources": [
          "sales.txt",
          "reports.txt",
          "product_info.txt"
        ],
        "other_resources": False
      },
      {
        "type": "customer",
  
        "two_factor_authentication": True,
        "multi_factor_authentication": True,
        "security_monitoring": True,
        "data_privacy_policy": True,
        "secure_file_uploads": False,
        "secure_file_uploads_policies": {
          "properties": {
            "secure_file_name": [".txt", ".csv", ".xlsx", ".pdf",".img",".png",".jpeg",".mp4"],
            "malware_scan": True,
            "audit_logging": True,
            "encryption": {
              "in_transit": True,
              "at_rest": True
            }
          }
        },
        "ssl_encryption_required": True,
        "permissions": ["read"],
        "explicite_allowed_resources": ["product_info.txt", "userId_info.txt"],
        "other_resources": False
      }
    ]
}

In [56]:
policy_score={
    "policies": [
      {
        "name": "admin",
        "properties": {
          "two_factor_authentication": 9,
          "multi_factor_authentication": 9,
          "security_monitoring": 10,
          "data_privacy_policy": 10,
          "secure_file_uploads": 8,
          "secure_file_uploads_policies": {
            "secure_file_name": 7,
            "malware_scan": 8,
            "audit_logging": 9,
            "sandboxing": 8,
            "encryption": {
              "in_transit": 10,
              "at_rest": 9
            }
          },
          "ssl_encryption_required": 8,
          "permissions": 9,
          "explicite_allowed_resources": 7,
          "other_resources": 6
        }
      },
      {
        "name": "employee",
        "properties": {
          "two_factor_authentication": 8,
          "multi_factor_authentication": 5,
          "security_monitoring": 7,
          "data_privacy_policy": 7,
          "secure_file_uploads": 6,
          "secure_file_uploads_policies": {
            "secure_file_name": 5,
            "malware_scan": 6,
            "audit_logging": 6,
            "encryption": {
              "in_transit": 8,
              "at_rest": 7
            }
          },
          "ssl_encryption_required": 7,
          "permissions": 6,
          "explicite_allowed_resources": 5,
          "other_resources": 3
        }
      },
      {
        "name": "customer",
        "properties": {
          "two_factor_authentication": 7,
          "security_monitoring": 6,
          "data_privacy_policy": 5,
          "secure_file_uploads": 3,
          "secure_file_uploads_policies": {
            "secure_file_name": 4,
            "malware_scan": 6,
            "audit_logging": 5,
            "encryption": {
              "in_transit": 7,
              "at_rest": 7
            }
          },
          "ssl_encryption_required": 5,
          "permissions": 4,
          "explicite_allowed_resources": 4,
          "other_resources": 3
        }
      }
    ]
  }
  

In [57]:
def check_policy_violation(instance):
    violations = {}
    k=instance["type"]
    desired_users = [user for user in rules["users"] if user["type"] == k]
 
    
    # Iterate over the rules for each user type
    for user_rule in desired_users:
            if user_rule["two_factor_authentication"] != instance["two_factor_authentication"]:
                violations["two_factor_authentication"]=instance["two_factor_authentication"]
            if user_rule["multi_factor_authentication"] != instance["multi_factor_authentication"]:
                violations["multi_factor_authentication"]=instance["multi_factor_authentication"]    
            if user_rule["security_monitoring"] != instance["security_monitoring"]:
                violations["security_monitoring"]=instance["security_monitoring"]
            if user_rule["data_privacy_policy"] != instance["data_privacy_policy"]:
                violations["data_privacy_policy"]=instance["data_privacy_policy"]
            
            
            
            for i in user_rule["secure_file_uploads_policies"]:
                for j in user_rule["secure_file_uploads_policies"][i]:
                    fname="secure_file_uploads_policies"
                    fname=fname+"__"+i
                    fname=fname+"__"+j

                    if(j=="secure_file_name"):
                      last_exe=instance[fname].split('.')
                      extension="."+last_exe[-1]
                      if extension not in [".txt", ".csv", ".xlsx", ".pdf",".img",".png",".jpeg",".mp4"]:
                          violations[fname]=instance[fname]
                    if(j=="malware_scan" and user_rule["secure_file_uploads_policies"][i][j]!=instance[fname]):
                        violations[fname]=instance[fname]
                    if(j=="audit_logging" and user_rule["secure_file_uploads_policies"][i][j]!=instance[fname]):
                        violations[fname]=instance[fname]
                    if(j=="encryption"):
                      for k in user_rule["secure_file_uploads_policies"][i][j]:
                        fname="secure_file_uploads_policies"
                        fname=fname+"__"+i
                        fname=fname+"__"+j
                        fname=fname+"__"+k
                        if(user_rule["secure_file_uploads_policies"][i][j][k]!=instance[fname]):
                            violations[fname]=instance[fname]
    
            if "ssl_encryption_required" in user_rule and user_rule["ssl_encryption_required"] != instance["ssl_encryption_required"]:
                violations["ssl_encryption_required"]=instance["ssl_encryption_required"]
            
            if "permissions" in user_rule and any(permission not in user_rule["permissions"] for permission in instance["permissions"]):
                violations["permissions"]=instance["permissions"]
            if "explicite_allowed_resources" in user_rule and instance["explicite_allowed_resources"] not in user_rule["explicite_allowed_resources"]:
                violations["explicite_allowed_resources"]=instance["explicite_allowed_resources"]
   
    return violations

def detect_user(instance):
    new_instance={}
    for key, value in instance.items():
        new_instance[key] = value
    
    violated_polices={}
    violations=check_policy_violation(instance)
    
    new_instance["violated_policies"]=violations
    
    return new_instance

def score_calculation(instance):
    instance = detect_user(instance)

    violated_policies = instance['violated_policies']
    score = 0

    user_type = instance['type']

    for policy in policy_score['policies']:
        if policy['name'] == user_type:
            for violation in violated_policies:
                if violation in policy['properties']:
                    score += policy['properties'][violation]



    return score

In [58]:
instance={"id": "64d0eb1a797d10ff6acb3de3", "client_id": "81.163.0.194", "date_time": "22/Jan/2019:12:43:44 +0330", "method": "GET", "request": "https://admin.flipkart.com/reports.txt/gallery?client_type=tier3&client_id=1513", "status_code": 451, "size": 439, "referer": "https://www.google.com/", "user_system_specs": "Android", "type": "customer", "two_factor_authentication": "false", "multi_factor_authentication": "true", "security_monitoring": "true", "data_privacy_policy": "false", "secure_file_uploads": "true", "secure_file_uploads_policies__properties__secure_file_name": "brave_sun.jpeg", "secure_file_uploads_policies__properties__malware_scan": "true", "secure_file_uploads_policies__properties__audit_logging": "true","secure_file_uploads_policies__properties__encryption__in_transit": "true", "secure_file_uploads_policies__properties__encryption__at_rest": "true", "ssl_encryption_required": "true", "permissions": "none", "explicite_allowed_resources": "product_info.txt", "other_resources": "true"}
score_calculation(instance)

27